# Set up software libraries
Authenticate and import as necessary.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=YikPSbnhRlWGfqjiMsVt4qXDH36BQgrQGN4l75fK_2w&tc=07VNtC48NN1iw3GFyC7Ddn3dd_PNs8ZV9mZJ_igx5UQ&cc=bPIN9oxdrlSDV9eNeFkMGF4VNn0uzm0M8ZaY9xsCBSs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VN_b9M-pKfisbEcR_b_jjANMLUE7ns-LtEiqysUV3uESC-ciUvNeeA

Successfully saved authorization token.


# Set variables

Insert details of the cloud storage bucket and folders, and specify the variables, size and shapes of inputs to the model.

In [ ]:
# Name of cloud storage bucket.
bucket = 'ee-surfacemelt'
model_folder = 'Models'

In [ ]:
# Name of model
model_name = 'AttUnet_500mShackletonTrained'
glob_model = 'gs://' + bucket + '/' + model_folder + '/' + model_name

# Prepare model for GEE predictions

###Training code package setup
It's necessary to create a Python package to hold the training code. Here we're going to get started with that by creating a folder for the package and adding an empty __init__.py file.

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(glob_model, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def. i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input_5:0": "array"}'
'{"StatefulPartitionedCall:0": "output"}'


## Run the EEifier
Use the command line to set your Cloud project and then run the eeifier.


In [ ]:
PROJECT = 'ee-iceshelf-gee4geo'
MODEL_DIR = glob_model

# Where to save the EEified model.
EEIFIED_DIR = 'gs://' + bucket + '/' + model_folder + '/' + model_name + '_EEified'

# Name of the AI Platform model to be hosted.
MODEL_NAME = model_name
# Version of the AI Platform model to be hosted.
VERSION_NAME = 'v0'

# This is a good region for hosting AI models.
REGION = 'us-central1'

In [ ]:
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://ee-surfacemelt/Models/AttUnet_500mShackletonTrained_SpatialPredictiveAnalyses_2March2023_TestMeltseason1617_EEified' is ready to be hosted in AI Platform.


## Deploy and host the EEified model on AI Platform
Before it's possible to get predictions from the trained and EEified model, it needs to be deployed on AI Platform.

In [ ]:
!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/ee-iceshelf-gee4geo/models/AttUnet_500mShackletonTrained_SpatialPredictiveAnalyses_2March2023_TestMeltseason1617].
Using endpoint [https://us-central1-ml.googleapis.com/]
